# Diccionario de Datos - Base de Datos PostgreSQL

Este documento contiene la descripción completa de todas las tablas, columnas, tipos de datos, constraints y relaciones de la base de datos del proyecto.


## 📋 Índice de Tablas

1. [usuarios](#1-usuarios)
2. [categorias](#2-categorias)
3. [productos](#3-productos)
4. [ordenes](#4-ordenes)
5. [detalle_ordenes](#5-detalle_ordenes)
6. [direcciones_envio](#6-direcciones_envio)
7. [carrito](#7-carrito)
8. [metodos_pago](#8-metodos_pago)
9. [ordenes_metodos_pago](#9-ordenes_metodos_pago)
10. [resenas_productos](#10-resenas_productos)
11. [historial_pagos](#11-historial_pagos)


## 1. usuarios

**Descripción:** Almacena la información de los usuarios del sistema.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `usuario_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único del usuario |
| `nombre` | VARCHAR(100) | NOT NULL | Nombre del usuario |
| `apellido` | VARCHAR(100) | NOT NULL | Apellido del usuario |
| `dni` | VARCHAR(20) | NOT NULL, UNIQUE | Documento Nacional de Identidad |
| `email` | VARCHAR(255) | NOT NULL, UNIQUE | Correo electrónico del usuario |
| `contraseña` | VARCHAR(255) | NOT NULL | Contraseña encriptada del usuario |
| `fecha_registro` | TIMESTAMP | DEFAULT now() | Fecha y hora de registro del usuario |

**Relaciones:**
- Referenciada por: `ordenes.usuario_id`, `direcciones_envio.usuario_id`, `carrito.usuario_id`, `resenas_productos.usuario_id`


## 2. categorias

**Descripción:** Almacena las categorías de productos disponibles en el sistema.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `categoria_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único de la categoría |
| `nombre` | VARCHAR(100) | NOT NULL | Nombre de la categoría |
| `descripcion` | VARCHAR(255) | NULL | Descripción detallada de la categoría |

**Relaciones:**
- Referenciada por: `productos.categoria_id`


## 3. productos

**Descripción:** Almacena la información de los productos disponibles para la venta.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `producto_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único del producto |
| `nombre` | VARCHAR(255) | NOT NULL | Nombre del producto |
| `descripcion` | TEXT | NULL | Descripción detallada del producto |
| `precio` | NUMERIC(10, 2) | NOT NULL, CHECK (precio >= 0) | Precio unitario del producto |
| `stock` | INTEGER | NOT NULL, CHECK (stock >= 0) | Cantidad disponible en inventario |
| `categoria_id` | INTEGER | FOREIGN KEY | Referencia a la categoría del producto |

**Constraints:**
- `check_precio_positivo`: Garantiza que el precio sea mayor o igual a 0
- `check_stock_positivo`: Garantiza que el stock sea mayor o igual a 0

**Relaciones:**
- `categoria_id` → `categorias.categoria_id`
- Referenciada por: `detalle_ordenes.producto_id`, `carrito.producto_id`, `resenas_productos.producto_id`


## 4. ordenes

**Descripción:** Almacena las órdenes de compra realizadas por los usuarios.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `orden_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único de la orden |
| `usuario_id` | INTEGER | FOREIGN KEY | Referencia al usuario que realizó la orden |
| `fecha_orden` | TIMESTAMP | DEFAULT now() | Fecha y hora en que se creó la orden |
| `total` | NUMERIC(10, 2) | NOT NULL, CHECK (total >= 0) | Monto total de la orden |
| `estado` | ENUM('estado_orden') | DEFAULT 'Pendiente' | Estado actual de la orden |

**Valores del Enum `estado_orden`:**
- `Pendiente`: Orden creada pero no procesada
- `Enviado`: Orden enviada al cliente
- `Completado`: Orden completada exitosamente
- `Cancelado`: Orden cancelada

**Constraints:**
- `check_total_positivo`: Garantiza que el total sea mayor o igual a 0

**Relaciones:**
- `usuario_id` → `usuarios.usuario_id`
- Referenciada por: `detalle_ordenes.orden_id`, `ordenes_metodos_pago.orden_id`, `historial_pagos.orden_id`


## 5. detalle_ordenes

**Descripción:** Almacena los detalles de cada producto incluido en una orden.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `detalle_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único del detalle |
| `orden_id` | INTEGER | FOREIGN KEY | Referencia a la orden |
| `producto_id` | INTEGER | FOREIGN KEY | Referencia al producto |
| `cantidad` | INTEGER | NOT NULL, CHECK (cantidad >= 0) | Cantidad de productos en el detalle |
| `precio_unitario` | NUMERIC(10, 2) | NOT NULL, CHECK (precio_unitario >= 0) | Precio unitario al momento de la orden |

**Constraints:**
- `check_cantidad_positiva`: Garantiza que la cantidad sea mayor o igual a 0
- `check_precio_unitario_positivo`: Garantiza que el precio unitario sea mayor o igual a 0

**Relaciones:**
- `orden_id` → `ordenes.orden_id`
- `producto_id` → `productos.producto_id`


## 6. direcciones_envio

**Descripción:** Almacena las direcciones de envío asociadas a los usuarios.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `direccion_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único de la dirección |
| `usuario_id` | INTEGER | FOREIGN KEY | Referencia al usuario propietario de la dirección |
| `calle` | VARCHAR(255) | NOT NULL | Nombre de la calle |
| `ciudad` | VARCHAR(100) | NOT NULL | Ciudad |
| `departamento` | VARCHAR(100) | NULL | Departamento o unidad |
| `provincia` | VARCHAR(100) | NULL | Provincia |
| `distrito` | VARCHAR(100) | NULL | Distrito |
| `estado` | VARCHAR(100) | NULL | Estado o región |
| `codigo_postal` | VARCHAR(20) | NULL | Código postal |
| `pais` | VARCHAR(100) | NOT NULL | País |

**Relaciones:**
- `usuario_id` → `usuarios.usuario_id`


## 7. carrito

**Descripción:** Almacena los productos que los usuarios han agregado a su carrito de compras.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `carrito_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único del item en el carrito |
| `usuario_id` | INTEGER | FOREIGN KEY | Referencia al usuario propietario del carrito |
| `producto_id` | INTEGER | FOREIGN KEY | Referencia al producto en el carrito |
| `cantidad` | INTEGER | NOT NULL, CHECK (cantidad >= 0) | Cantidad del producto en el carrito |
| `fecha_agregado` | TIMESTAMP | DEFAULT now() | Fecha y hora en que se agregó el producto |

**Constraints:**
- `check_cantidad_carrito_positiva`: Garantiza que la cantidad sea mayor o igual a 0

**Relaciones:**
- `usuario_id` → `usuarios.usuario_id`
- `producto_id` → `productos.producto_id`


## 8. metodos_pago

**Descripción:** Almacena los métodos de pago disponibles en el sistema.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `metodo_pago_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único del método de pago |
| `nombre` | VARCHAR(100) | NOT NULL | Nombre del método de pago |
| `descripcion` | VARCHAR(255) | NULL | Descripción del método de pago |

**Relaciones:**
- Referenciada por: `ordenes_metodos_pago.metodo_pago_id`, `historial_pagos.metodo_pago_id`


## 9. ordenes_metodos_pago

**Descripción:** Relación entre órdenes y métodos de pago, almacena el monto pagado con cada método.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `orden_metodo_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único del registro |
| `orden_id` | INTEGER | FOREIGN KEY | Referencia a la orden |
| `metodo_pago_id` | INTEGER | FOREIGN KEY | Referencia al método de pago utilizado |
| `monto_pagado` | NUMERIC(10, 2) | NOT NULL, CHECK (monto_pagado >= 0) | Monto pagado con este método |

**Constraints:**
- `check_monto_pagado_positivo`: Garantiza que el monto pagado sea mayor o igual a 0

**Relaciones:**
- `orden_id` → `ordenes.orden_id`
- `metodo_pago_id` → `metodos_pago.metodo_pago_id`


## 10. resenas_productos

**Descripción:** Almacena las reseñas y calificaciones que los usuarios hacen sobre los productos.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `resena_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único de la reseña |
| `usuario_id` | INTEGER | FOREIGN KEY | Referencia al usuario que escribió la reseña |
| `producto_id` | INTEGER | FOREIGN KEY | Referencia al producto reseñado |
| `calificacion` | INTEGER | CHECK (calificacion >= 1 AND calificacion <= 5) | Calificación del producto (1-5 estrellas) |
| `comentario` | TEXT | NULL | Comentario o reseña escrita por el usuario |
| `fecha` | TIMESTAMP | DEFAULT now() | Fecha y hora en que se publicó la reseña |

**Constraints:**
- `check_calificacion_rango`: Garantiza que la calificación esté entre 1 y 5

**Relaciones:**
- `usuario_id` → `usuarios.usuario_id`
- `producto_id` → `productos.producto_id`


## 11. historial_pagos

**Descripción:** Almacena el historial de todos los pagos realizados en el sistema.

| Columna | Tipo | Restricciones | Descripción |
|---------|------|---------------|-------------|
| `pago_id` | INTEGER | PRIMARY KEY, AUTO_INCREMENT | Identificador único del pago |
| `orden_id` | INTEGER | FOREIGN KEY | Referencia a la orden asociada al pago |
| `metodo_pago_id` | INTEGER | FOREIGN KEY | Referencia al método de pago utilizado |
| `monto` | NUMERIC(10, 2) | NOT NULL, CHECK (monto >= 0) | Monto del pago |
| `fecha_pago` | TIMESTAMP | DEFAULT now() | Fecha y hora en que se realizó el pago |
| `estado_pago` | ENUM('estado_pago') | DEFAULT 'Procesando' | Estado actual del pago |

**Valores del Enum `estado_pago`:**
- `Procesando`: Pago en proceso de validación
- `Pagado`: Pago completado exitosamente
- `Fallido`: Pago fallido o rechazado
- `Reembolsado`: Pago reembolsado al cliente

**Constraints:**
- `check_monto_positivo`: Garantiza que el monto sea mayor o igual a 0

**Relaciones:**
- `orden_id` → `ordenes.orden_id`
- `metodo_pago_id` → `metodos_pago.metodo_pago_id`


## 📊 Resumen de Tipos de Datos

### Tipos Numéricos
- **INTEGER**: Números enteros (usado para IDs, cantidades, stock, calificaciones)
- **NUMERIC(10, 2)**: Números decimales con precisión de 10 dígitos y 2 decimales (usado para precios, montos, totales)

### Tipos de Texto
- **VARCHAR(n)**: Cadena de caracteres de longitud variable hasta n caracteres
- **TEXT**: Cadena de caracteres de longitud ilimitada (usado para descripciones largas y comentarios)

### Tipos de Fecha y Hora
- **TIMESTAMP**: Fecha y hora con zona horaria (usado para fechas de registro, órdenes, pagos)

### Tipos Especiales
- **ENUM**: Tipo enumerado nativo de PostgreSQL (usado para estados de orden y pago)
- **SERIAL**: Entero auto-incremental (equivalente a INTEGER con AUTO_INCREMENT)


## 🔗 Diagrama de Relaciones

```
usuarios (1) ──< (N) ordenes
usuarios (1) ──< (N) direcciones_envio
usuarios (1) ──< (N) carrito
usuarios (1) ──< (N) resenas_productos

categorias (1) ──< (N) productos

productos (1) ──< (N) detalle_ordenes
productos (1) ──< (N) carrito
productos (1) ──< (N) resenas_productos

ordenes (1) ──< (N) detalle_ordenes
ordenes (1) ──< (N) ordenes_metodos_pago
ordenes (1) ──< (N) historial_pagos

metodos_pago (1) ──< (N) ordenes_metodos_pago
metodos_pago (1) ──< (N) historial_pagos
```

**Leyenda:**
- `(1)` = Uno
- `(N)` = Muchos
- `──<` = Relación uno a muchos


## ✅ Constraints y Validaciones

### CheckConstraints (Validaciones Numéricas)

| Tabla | Constraint | Validación |
|-------|-----------|------------|
| `productos` | `check_precio_positivo` | `precio >= 0` |
| `productos` | `check_stock_positivo` | `stock >= 0` |
| `ordenes` | `check_total_positivo` | `total >= 0` |
| `detalle_ordenes` | `check_cantidad_positiva` | `cantidad >= 0` |
| `detalle_ordenes` | `check_precio_unitario_positivo` | `precio_unitario >= 0` |
| `carrito` | `check_cantidad_carrito_positiva` | `cantidad >= 0` |
| `ordenes_metodos_pago` | `check_monto_pagado_positivo` | `monto_pagado >= 0` |
| `historial_pagos` | `check_monto_positivo` | `monto >= 0` |
| `resenas_productos` | `check_calificacion_rango` | `calificacion >= 1 AND calificacion <= 5` |

### Enums Nativos de PostgreSQL

| Enum | Valores | Tabla | Columna |
|------|---------|-------|---------|
| `estado_orden` | Pendiente, Enviado, Completado, Cancelado | `ordenes` | `estado` |
| `estado_pago` | Procesando, Pagado, Fallido, Reembolsado | `historial_pagos` | `estado_pago` |

### Foreign Keys

Todas las relaciones entre tablas están implementadas mediante Foreign Keys para garantizar la integridad referencial.


## 📝 Notas Adicionales

### Valores por Defecto
- Los campos `fecha_registro`, `fecha_orden`, `fecha_agregado`, `fecha` y `fecha_pago` utilizan `DEFAULT now()` para establecer automáticamente la fecha y hora actual.
- El campo `estado` en `ordenes` tiene valor por defecto `'Pendiente'`.
- El campo `estado_pago` en `historial_pagos` tiene valor por defecto `'Procesando'`.

### Campos Únicos
- `usuarios.dni`: Cada DNI debe ser único en el sistema
- `usuarios.email`: Cada email debe ser único en el sistema

### Campos Obligatorios (NOT NULL)
Todos los campos marcados como `NOT NULL` son obligatorios y deben tener un valor al insertar registros.

### Auto-incremento
Todos los campos `PRIMARY KEY` utilizan `AUTO_INCREMENT` (SERIAL en PostgreSQL) para generar automáticamente valores únicos secuenciales.
